In [1]:
import json
import asyncio
from datetime import datetime
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup

In [2]:
URL = "https://sports2.tippmixpro.hu/hu/esemenyek/11/labdarugas/vilag/valogatott-merkozes/lengyelorszag-uj-zeland/282705733837164544/all"

In [3]:
with open('tipmix.txt', encoding='utf-8') as f:
    kellenek = [line.strip() for line in f]

In [16]:
def parse_html(html, kell=None):
    """
    Szinkron feldolgozás.
    Adatkinyerés
    """

    if kell is None:
        kell = set()

    markets = {}

    soup = BeautifulSoup(html, "html.parser")

    container = soup.select_one(".MarketGroupsItem")

    if container is None:
        # nincs fő konténer -> üres eredmény
        return markets


    for article in container.select("article"):
        
        soradatok = {}

        header_el = article.select_one(".Market__CollapseText")
        header_text = header_el.get_text(strip=True) if header_el else ""

        #if header_text not in kell:
        #    continue

        oddsgroup = article.select("ul.Market__OddsGroups")  # li elemek listája ezen az ul tag-en belül

        print(header_text)

        if len(oddsgroup) == 0:  
            print('bezárt menü')
            continue

        ligroup = oddsgroup[0].find_all('li', recursive=False)
        print('ligroup (sorok) száma:', len(ligroup))
    
        alcimCount = len(oddsgroup[0].find_all("li", class_="Market__OddsGroupTitle"))
        print('alcímek száma:', alcimCount)

        
        # ha nincsen alcím és nincsen header
        if len(ligroup) == 1 and alcimCount == 0:  
            group = ligroup[0] # egy eleme van akkor a gorup legyen az egyetlen elem

            for alcim in group.find_all("li"):
                oszlop = [i.get_text(strip=True) for i in alcim.find_all('span')]
                if len(oszlop) < 2:
                    print('zárolt oszlop!')
                    continue
                soradatok[oszlop[0]] = oszlop[1]     # oszlop 0 itt lehet névcsere
                     
        # ha van alcím

        # a header száma egyezik a lingroup számmal
        elif alcimCount == len(ligroup):
            print('egyezik a lingroup és a header szám (kihagyás)!!!!')
            print()
            continue

        # egy header van (header van az első helyen)
        elif "Market__HeadersWrapper" in ligroup[0].get("class", []):
            header_titles = [i.get_text(strip=True) for i in ligroup[0].find_all('li')]
            #print('\t\t header:', header_titles)

            # ha van header akkor még el kell dönteni, hogy van-e Market__OddsGroupTitle
            if ligroup[1].select_one("li.Market__OddsGroupTitle"):
                
                for li in ligroup[1:]:
                    ossdd = [ods.get_text(strip=True) for ods in li.find_all('li')]
                    soradatok[ossdd[0]] = {header_titles[0]:ossdd[1], header_titles[2]:ossdd[2]}

            # nincsen header, csak sok alcím
            else:
                print('Nincsen header %%%%%')
                for sor in ligroup[1:]:
                    for i, oszlop in enumerate(sor.find_all('li')):
                        if len(oszlop.find_all('span')) != 2:
                            continue

                        text, odds = [m.get_text(strip=True) for m in oszlop.find_all('span')]
                        #print(header_titles[i], text, odds)
                        #print(text, odds)
                        soradatok[text] = odds
                        
        markets[header_text] = soradatok
        print(soradatok)
        print()
        
    return markets

In [17]:
m = parse_html(html, kell=kellenek)

1X2 - Rendes játékidő
ligroup (sorok) száma: 1
alcímek száma: 0
{'Lengyelország': '1,52', 'Döntetlen': '4,40', 'Új-Zéland': '6,75'}

Mindkét csapat szerez gólt - Rendes játékidő
ligroup (sorok) száma: 1
alcímek száma: 0
{'Igen': '2,06', 'Nem': '1,77'}

1X2 + Mindkét csapat szerez gólt - Rendes játékidő
ligroup (sorok) száma: 4
alcímek száma: 0
Nincsen header %%%%%
{'Hazai és Nem': '2,25', 'Hazai és Igen': '3,40', 'Döntetlen és Igen': '5,75', 'Döntetlen és Nem': '10,50', 'Vendég és Nem': '9,25', 'Vendég és Igen': '11,50'}

Gólszám - Rendes játékidő
ligroup (sorok) száma: 20
alcímek száma: 19
{'0,5': {'Több, mint': '1,05', 'Kevesebb, mint': '10,00'}, '1': {'Több, mint': '1,07', 'Kevesebb, mint': '8,25'}, '1,25': {'Több, mint': '1,18', 'Kevesebb, mint': '4,75'}, '1,5': {'Több, mint': '1,30', 'Kevesebb, mint': '3,55'}, '1,75': {'Több, mint': '1,36', 'Kevesebb, mint': '3,10'}, '2': {'Több, mint': '1,45', 'Kevesebb, mint': '2,70'}, '2,25': {'Több, mint': '1,67', 'Kevesebb, mint': '2,20'}, '2

In [18]:
m

{'1X2 - Rendes játékidő': {'Lengyelország': '1,52',
  'Döntetlen': '4,40',
  'Új-Zéland': '6,75'},
 'Mindkét csapat szerez gólt - Rendes játékidő': {'Igen': '2,06',
  'Nem': '1,77'},
 '1X2 + Mindkét csapat szerez gólt - Rendes játékidő': {'Hazai és Nem': '2,25',
  'Hazai és Igen': '3,40',
  'Döntetlen és Igen': '5,75',
  'Döntetlen és Nem': '10,50',
  'Vendég és Nem': '9,25',
  'Vendég és Igen': '11,50'},
 'Gólszám - Rendes játékidő': {'Páratlan': '1,80', 'Páros': '1,80'},
 'Lengyelország gólszám - Rendes játékidő': {'0,5': {'Több, mint': '1,12',
   'Kevesebb, mint': '5,50'},
  '0,75': {'Több, mint': '1,13', 'Kevesebb, mint': '5,00'},
  '1': {'Több, mint': '1,18', 'Kevesebb, mint': '4,30'},
  '1,25': {'Több, mint': '1,41', 'Kevesebb, mint': '2,70'},
  '1,5': {'Több, mint': '1,62', 'Kevesebb, mint': '2,14'},
  '1,75': {'Több, mint': '1,84', 'Kevesebb, mint': '1,88'},
  '2': {'Több, mint': '2,19', 'Kevesebb, mint': '1,61'},
  '2,25': {'Több, mint': '2,57', 'Kevesebb, mint': '1,44'},
  '2

In [7]:
html = ''
# ------------- ASYNC LOOP -------------
async def run_scraper(url, headless=True, interval_sec=60, iterations=None, parser_fn=parse_html, on_result=None):
    """
    - Megnyitja a böngészőt és az oldalt.
    - Ciklusban (percenként) lekéri az oldal tartalmát.
    - A HTML-t átadja egy SZINKRON parser_fn-nek.
    - Ha adsz on_result callbacket (szinkron), annak átadja az eredményt.
    - iterations: ha None -> végtelen ciklus, ha szám -> ennyiszer fut.
    """
    global html
    
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=headless)
        context = await browser.new_context()
        page = await context.new_page()

        try:
            # első betöltés
            await page.goto(url, timeout=45000)
            await page.wait_for_selector(".MarketGroupsItem", timeout=15000)


            count = 0
            while True:
                # frissítés (reload), vagy újra goto – ahogy jobban esik
                try:
                    await page.reload(timeout=45000)
                    await page.wait_for_selector(".MarketGroupsItem", timeout=15000)
                    
                except Exception:
                    # ha reload nem ok, próbálkozhatsz goto-val
                    await page.goto(url, wait_until="domcontentloaded", timeout=45000)

                # HTML kiolvasása (ASYNC), majd átadjuk a SZINKRON parsernek
                html = await page.content()
                result = parser_fn(html)

                # ha adtál callbacket, itt megkapja az eredményt (SZINKRON)
                if on_result:
                    on_result(result)
                else:
                    # default: csak kiírjuk (váz)
                    print(result)

                count += 1
                if iterations is not None and count >= iterations:
                    break

                await asyncio.sleep(interval_sec)

        finally:
            await context.close()
            await browser.close()

In [8]:
await run_scraper(URL, headless=False, interval_sec=60, iterations=1)

1X2 - Rendes játékidő
ligroup (sorok) száma: 1
alcímek száma: 0
{'Lengyelország': '1,52', 'Döntetlen': '4,40', 'Új-Zéland': '6,75'}

Mindkét csapat szerez gólt - Rendes játékidő
ligroup (sorok) száma: 1
alcímek száma: 0
{'Igen': '2,06', 'Nem': '1,77'}

1X2 + Mindkét csapat szerez gólt - Rendes játékidő
ligroup (sorok) száma: 4
alcímek száma: 0
Nincsen header %%%%%
{'Hazai és Nem': '2,25', 'Hazai és Igen': '3,40', 'Döntetlen és Igen': '5,75', 'Döntetlen és Nem': '10,50', 'Vendég és Nem': '9,25', 'Vendég és Igen': '11,50'}

Gólszám - Rendes játékidő
ligroup (sorok) száma: 20
alcímek száma: 19
{'0,5': {'Több, mint': '1,05', 'Kevesebb, mint': '10,00'}, '1': {'Több, mint': '1,07', 'Kevesebb, mint': '8,25'}, '1,25': {'Több, mint': '1,18', 'Kevesebb, mint': '4,75'}, '1,5': {'Több, mint': '1,30', 'Kevesebb, mint': '3,55'}, '1,75': {'Több, mint': '1,36', 'Kevesebb, mint': '3,10'}, '2': {'Több, mint': '1,45', 'Kevesebb, mint': '2,70'}, '2,25': {'Több, mint': '1,67', 'Kevesebb, mint': '2,20'}, '2

IndexError: list index out of range